In [1]:
from phe import paillier
import numpy as np
from openpyxl import load_workbook
import time

In [2]:
start_time = time.time()

In [3]:
wb = load_workbook("marketing_case2.xlsx")
ws1 = wb["A1"]
ws2 = wb["B1"]
n = 10000

# Bank of data

In [4]:
class Bank:
  def __init__(self, public_key):
    self.records = []
    for i in range(2,n):
        if i%5000 == 0:
            print(i)
        phone_number = ws1["A" + str(i)].value
        spending = np.random.uniform(0, 100)
        timestamp = np.random.randint(0, 100)
        self.records.append((phone_number, public_key.encrypt(spending), timestamp))

# Advertisement Hub

In [5]:
class AdvertisementHub:
     def __init__(self, public_key):
         self.records = []
         for i in range(2, n):
             if i%5000 == 0:
                print(i)
             phone_number = ws2["A" + str(i)].value
             advertisement_views = np.random.randint(0, 10)
             timestamp = np.random.randint(0, 100)
             self.records.append((phone_number, public_key.encrypt(advertisement_views), timestamp))

# PS3I

In [6]:
class PS3I:
        def __init__(self, bank, advertisement_hub, public_key):
            self.bank = bank
            self.advertisement_hub = advertisement_hub
            self.public_key = public_key
            # self.encryptor = paillier.PaillierEncryptor(self.public_key)

        def encrypt_records(self, records):
            encrypted_record = self.public_key.encrypt(records)
            return encrypted_record

        def decrypt_record(self, encrypted_record):
            record = [val.decrypt() for val in encrypted_record]
            return record

        def intersect(self):
            bank_records = self.bank.records
            ad_records = self.advertisement_hub.records
            result = []
            for bank_record in bank_records:
                for ad_record in ad_records:
                    if bank_record[0] == ad_record[0]:
                        result.append((self.encrypt_records(bank_record[0]), bank_record[1], ad_record[1], bank_record[2]))
            return bank_records

        def calculate_risk(self, epsilon):
            intersection_size = len(self.intersect())
            bank_size = len(self.bank.records)
            advertisement_hub_size = len(self.advertisement_hub.records)
            privacy_budget = bank_size * advertisement_hub_size / intersection_size
            risk_level = epsilon / privacy_budget
            return risk_level

 # Example usage:

In [7]:
public_key, private_key = paillier.generate_paillier_keypair()
bank = Bank(public_key)
advertisement_hub = AdvertisementHub(public_key)
ps3i = PS3I(bank, advertisement_hub, public_key)

5000
5000


# Perform homomorphic addition and multiplication to calculate the sum of spending and advertisement views

In [8]:
spending_sum = public_key.encrypt(0)
ad_views_sum = public_key.encrypt(0)
for bank_record in bank.records:
     spending_sum += bank_record[1]
for ad_record in advertisement_hub.records:
     ad_views_sum += ad_record[1]

# Decrypt the homomorphically computed results

In [9]:
spending_sum = private_key.decrypt(spending_sum)
ad_views_sum = private_key.decrypt(ad_views_sum)

intersection = ps3i.intersect()
risk_level = ps3i.calculate_risk(0.1)
end_time = time.time()
value = format(float(risk_level),".10f")
print(f"Intersection size = {len(intersection)}")
print(f"Risk level = {risk_level} = {value}")
print(f"time = {(end_time - start_time) / 60}min")

Intersection size = 9998
Risk level = 1.0002000400080017e-05 = 0.0000100020
time = 149.2117377082507min
